In [1]:
import pandas as pd
import re
import os

In [2]:
# Update of multiple hcs file

import pandas as pd
import os

# Ask user if they have multiple HCS files
multiple_files = input("Do you have multiple HCS files to process? (yes/no): ").lower().strip()

all_hcs_data = []

def process_file(file_path):
    """Process a single file (CSV, XLS, or HTML) and return a DataFrame."""
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.html'):
            with open(file_path, 'r', encoding='utf-8') as file:
                html_content = file.read()
            df_list = pd.read_html(html_content)
            df = df_list[0]
            # Set first row as column names for HTML files
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
        else:
            raise ValueError("Unsupported file format. Use CSV, XLS/XLSX, or HTML.")
        return df
    except Exception as e:
        print(f"⚠️ Error processing {file_path}: {str(e)}")
        return None

if multiple_files == 'yes':
    # Ask for multiple file paths
    hcs_paths = []
    while True:
        path = input("Please enter the path to an HCS file (html/xls/csv) (or 'done' to finish): ")
        if path.lower() == 'done':
            break
        if os.path.exists(path):
            hcs_paths.append(path)
        else:
            print("Error: The file does not exist. Please check the path and try again.")

    if not hcs_paths:
        print("Error: No valid files provided. Exiting.")
    else:
        # Process each HCS file
        for hcs_path in hcs_paths:
            hcs = process_file(hcs_path)
            if hcs is not None:
                all_hcs_data.append(hcs)
                print(f"✅ Successfully processed: {hcs_path}")

        # Combine all dataframes
        if all_hcs_data:
            combined_hcs = pd.concat(all_hcs_data, ignore_index=True)
            print(f"\n✅ Combined {len(all_hcs_data)} HCS files")
            print("First few rows of combined data:")
            print(combined_hcs.head())
            # Option to save
            save = input("Would you like to save the combined data to a CSV file? (yes/no): ").lower().strip()
            if save == 'yes':
                output_path = input("Enter the output file path (e.g., output.csv): ")
                combined_hcs.to_csv(output_path, index=False)
                print(f"✅ Saved to {output_path}")
        else:
            print("No data was successfully processed.")

else:
    # Single file processing
    hcs_path = input("Please enter the path to your HCS file (html/xls/csv): ")
    if not os.path.exists(hcs_path):
        print("Error: The file does not exist. Please check the path and try again.")
    else:
        hcs = process_file(hcs_path)
        if hcs is not None:
            print(f"\n✅ Successfully processed: {hcs_path}")
            print("First few rows of data:")
            print(hcs.head())
            print(f"Total rows: {len(hcs)}")
            # Option to save
            save = input("Would you like to save the data to a CSV file? (yes/no): ").lower().strip()
            if save == 'yes':
                output_path = input("Enter the output file path (e.g., output.csv): ")
                hcs.to_csv(output_path, index=False)
                print(f"✅ Saved to {output_path}")

Do you have multiple HCS files to process? (yes/no): yes
Please enter the path to an HCS file (html/xls/csv) (or 'done' to finish): today2.xlsx
Please enter the path to an HCS file (html/xls/csv) (or 'done' to finish): done
✅ Successfully processed: today2.xlsx

✅ Combined 1 HCS files
First few rows of combined data:
   ISSTYPE       CARD_NUMBER             CRDH_NAME         ATM_ACCT  \
0  REPLACE  4753960000746392           IJAJ HUSSEN  414701406417018   
1  REPLACE  4753960000742094  TEJLAXMI RAJBHANDARI  210200053905015   
2  REPLACE  4753960000742102  PURNA BAHADUR KHADGI  210201108720016   
3  REPLACE  4753960000742193    SUKRA P. RANJITKAR  210001312381019   
4  REPLACE  4753960000742219        BISHNU B SINGH  726105035139015   

     ISS_DATE  EXPIR_DATE  CARD_ID  
0  2025-03-24  2029-03-24  1308014  
1  2025-03-24  2029-03-24  1306670  
2  2025-03-24  2029-03-24  1306671  
3  2025-03-24  2029-03-24  1306678  
4  2025-03-24  2029-03-24  1306680  
Would you like to save the combi

In [3]:
import os
import re
import pandas as pd

def normalize_name(name):
    """Normalize a name by converting to uppercase, removing extra spaces, and splitting into components."""
    if pd.isna(name) or name == "NaN":
        return {"first": "", "middle": "", "surname": ""}
    # Remove extra spaces and convert to uppercase
    name = " ".join(name.split()).upper()
    parts = name.split()
    
    if len(parts) == 1:
        return {"first": parts[0], "middle": "", "surname": ""}
    elif len(parts) == 2:
        return {"first": parts[0], "middle": "", "surname": parts[1]}
    else:
        return {"first": parts[0], "middle": " ".join(parts[1:-1]), "surname": parts[-1]}

def process_hcs_file(file_path):
    """Process an HCS file (HTML, CSV, or XLS/XLSX) and return a DataFrame."""
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.html'):
            with open(file_path, 'r', encoding='utf-8') as file:
                html_content = file.read()
            df_list = pd.read_html(html_content)
            df = df_list[0]
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
        else:
            raise ValueError("Unsupported file format. Use CSV, XLS/XLSX, or HTML.")
        return df
    except Exception as e:
        print(f"⚠️ Error processing {file_path}: {str(e)}")
        return None

# --- Process Extracted Names ---
multiple_txt_files = input("Do you have multiple text files for name extraction? (yes/no): ").lower().strip()
all_extracted_names = []

if multiple_txt_files == 'yes':
    txt_paths = []
    while True:
        path = input("Please enter the path to a text file (or 'done' to finish): ")
        if path.lower() == 'done':
            break
        if os.path.exists(path):
            txt_paths.append(path)
        else:
            print("Error: File does not exist. Please check the path and try again.")

    if not txt_paths:
        print("Error: No valid text files provided. Exiting.")
    else:
        for txt_path in txt_paths:
            try:
                with open(txt_path, 'r', encoding='utf-8') as txt_file:
                    for line in txt_file:
                        matches = re.findall(r'NPR\s{0,4}([A-Z][A-Z.\s]+)', line)
                        if matches:
                            for match in matches:
                                all_extracted_names.append(match.strip())
                        else:
                            all_extracted_names.append("NaN")
                print(f"✅ Successfully processed: {txt_path}")
            except Exception as e:
                print(f"⚠️ Error processing {txt_path}: {str(e)}")
else:
    txt_path = input("Please enter the path to your text file: ")
    if not os.path.exists(txt_path):
        print("Error: File does not exist. Please check the path and try again.")
    else:
        try:
            with open(txt_path, 'r', encoding='utf-8') as txt_file:
                for line in txt_file:
                    matches = re.findall(r'NPR\s{0,4}([A-Z][A-Z.\s]+)', line)
                    if matches:
                        for match in matches:
                            all_extracted_names.append(match.strip())
                    else:
                        all_extracted_names.append("NaN")
            print(f"✅ Successfully processed: {txt_path}")
        except Exception as e:
            print(f"⚠️ Error processing {txt_path}: {str(e)}")

if not all_extracted_names:
    print("Error: No names extracted. Exiting.")
    exit()

extracted_df = pd.DataFrame({'EXTRACTED_NAME': all_extracted_names})

# --- Process HCS Files ---
multiple_hcs_files = input("Do you have multiple HCS files to process? (yes/no): ").lower().strip()
all_hcs_data = []

if multiple_hcs_files == 'yes':
    hcs_paths = []
    while True:
        path = input("Please enter the path to an HCS file (html/xls/csv) (or 'done' to finish): ")
        if path.lower() == 'done':
            break
        if os.path.exists(path):
            hcs_paths.append(path)
        else:
            print("Error: File does not exist. Please check the path and try again.")

    if not hcs_paths:
        print("Error: No valid HCS files provided. Exiting.")
    else:
        for hcs_path in hcs_paths:
            hcs = process_hcs_file(hcs_path)
            if hcs is not None:
                all_hcs_data.append(hcs)
                print(f"✅ Successfully processed: {hcs_path}")
        if all_hcs_data:
            hcs_df = pd.concat(all_hcs_data, ignore_index=True)
        else:
            print("Error: No HCS data successfully processed. Exiting.")
            exit()
else:
    hcs_path = input("Please enter the path to your HCS file (html/xls/csv): ")
    if not os.path.exists(hcs_path):
        print("Error: File does not exist. Please check the path and try again.")
    else:
        hcs_df = process_hcs_file(hcs_path)
        if hcs_df is None:
            print("Error: HCS file processing failed. Exiting.")
            exit()
        print(f"✅ Successfully processed: {hcs_path}")

# --- Name Matching ---
hcs_name_col = 'CRDH_NAME'  # Replace with actual column name if different

# Normalize names
hcs_df['norm'] = hcs_df[hcs_name_col].apply(normalize_name)
extracted_df['norm'] = extracted_df['EXTRACTED_NAME'].apply(normalize_name)

# Create matching keys
hcs_df['match_key'] = hcs_df['norm'].apply(lambda x: f"{x['first']}|{x['middle']}|{x['surname']}")
extracted_df['match_key'] = extracted_df['norm'].apply(lambda x: f"{x['first']}|{x['middle']}|{x['surname']}")

# Use extracted_df as base (smaller set)
matched_df = extracted_df.merge(
    hcs_df,
    how='left',
    on='match_key',
    suffixes=('_extracted', '_hcs')
)

# Add extracted name to matched rows
matched_df['MATCHED_EXTRACTED_NAME'] = matched_df['EXTRACTED_NAME']

# Clean up: keep only necessary columns and remove unmatched rows
final_df = matched_df.dropna(subset=[hcs_name_col])  # Drop rows where HCS data wasn't matched
final_df = final_df.drop(columns=['norm_extracted', 'norm_hcs', 'match_key'])

# Results
print(f"\nOriginal HCS rows: {len(hcs_df)}")
print(f"Extracted names: {len(extracted_df)}")
print(f"Final matched rows: {len(final_df)}")
print("\nFirst few rows of final DataFrame:")
print(final_df.head())

# Optionally save to CSV
save = input("Would you like to save the results to a CSV file? (yes/no): ").lower().strip()
if save == 'yes':
    output_path = input("Enter the output file path (e.g., matched_output.csv): ")
    final_df.to_csv(output_path, index=False)
    print(f"✅ Results saved to {output_path}")
else:
    print("Results not saved.")

Do you have multiple text files for name extraction? (yes/no):  yes
Please enter the path to a text file (or 'done' to finish):  today


Error: File does not exist. Please check the path and try again.


Please enter the path to a text file (or 'done' to finish):  today.txt
Please enter the path to a text file (or 'done' to finish):  today1.txt
Please enter the path to a text file (or 'done' to finish):  today2.txt
Please enter the path to a text file (or 'done' to finish):  today3.txt
Please enter the path to a text file (or 'done' to finish):  done


✅ Successfully processed: today.txt
✅ Successfully processed: today1.txt
✅ Successfully processed: today2.txt
✅ Successfully processed: today3.txt


Do you have multiple HCS files to process? (yes/no):  no
Please enter the path to your HCS file (html/xls/csv):  hcs.xlsx


✅ Successfully processed: hcs.xlsx

Original HCS rows: 237
Extracted names: 1473
Final matched rows: 234

First few rows of final DataFrame:
        EXTRACTED_NAME  ISSTYPE   CARD_NUMBER           CRDH_NAME  \
16    PRAKASH B ROKAYA  REPLACE  4.753960e+15    PRAKASH B ROKAYA   
17     PAN SINGH DHAMI  REPLACE  4.753960e+15     PAN SINGH DHAMI   
24  NAR BAHADUR BOHARA  REPLACE  4.753960e+15  NAR BAHADUR BOHARA   
33         SABIN DAHAL  REPLACE  4.753960e+15         SABIN DAHAL   
35   SUSHMA P ADHIKARI  REPLACE  4.753960e+15   SUSHMA P ADHIKARI   

        ATM_ACCT    ISS_DATE  EXPIR_DATE    CARD_ID MATCHED_EXTRACTED_NAME  
16  8.234051e+14  2025-03-24  2029-03-24  1307676.0       PRAKASH B ROKAYA  
17  8.234017e+14  2025-03-24  2029-03-24  1307677.0        PAN SINGH DHAMI  
24  8.234016e+14  2025-03-24  2029-03-24  1307684.0     NAR BAHADUR BOHARA  
33  2.126052e+14  2025-03-24  2029-03-24  1307693.0            SABIN DAHAL  
35  4.153054e+14  2025-03-24  2029-03-24  1307717.0      SU

Would you like to save the results to a CSV file? (yes/no):  no


Results not saved.


In [5]:
# find duplication in HCS file
# After creating hcs_df but before merging
duplicate_hcs_names = hcs_df['match_key'].duplicated(keep=False)
print(f"Number of HCS rows with duplicate match_keys: {duplicate_hcs_names.sum()}")
print(f"Unique match_keys in HCS with duplicates: {hcs_df[duplicate_hcs_names]['match_key'].nunique()}")

# After the merge, before final_df
print(f"Rows in matched_df before dropping NaN: {len(matched_df)}")
duplicate_matches = matched_df['match_key'].duplicated(keep=False)
print(f"Number of matched rows with duplicate match_keys: {duplicate_matches.sum()}")

Number of HCS rows with duplicate match_keys: 0
Unique match_keys in HCS with duplicates: 0
Rows in matched_df before dropping NaN: 1473
Number of matched rows with duplicate match_keys: 16


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime, timedelta

def normalize_name(name):
    """Normalize a name by converting to uppercase, removing extra spaces, and splitting into components"""
    if pd.isna(name) or name == "NaN":
        return {"first": "", "middle": "", "surname": ""}
    name = " ".join(name.split()).upper()
    parts = name.split()
    if len(parts) == 1:
        return {"first": parts[0], "middle": "", "surname": ""}
    elif len(parts) == 2:
        return {"first": parts[0], "middle": "", "surname": parts[1]}
    else:
        return {"first": parts[0], "middle": " ".join(parts[1:-1]), "surname": parts[-1]}

# --- Process Extracted Names ---
multiple_txt_files = input("Do you have multiple text files for name extraction? (yes/no): ").lower().strip()
all_extracted_names = []
line_number = 0

if multiple_txt_files == 'yes':
    txt_paths = []
    while True:
        path = input("Please enter the path to a text file (or 'done' to finish): ")
        if path.lower() == 'done':
            break
        if os.path.exists(path):
            txt_paths.append(path)
    for txt_path in txt_paths:
        with open(txt_path, 'r', encoding='utf-8') as txt_file:
            for line in txt_file:
                line_number += 1
                matches = re.findall(r'NPR\s{0,4}([A-Z][A-Z.\s]+)', line)
                if matches:
                    for match in matches:
                        all_extracted_names.append(match.strip())
                else:
                    all_extracted_names.append("NaN")
else:
    txt_path = input("Please enter the path to your text file: ")
    if os.path.exists(txt_path):
        with open(txt_path, 'r', encoding='utf-8') as txt_file:
            for line in txt_file:
                line_number += 1
                matches = re.findall(r'NPR\s{0,4}([A-Z][A-Z.\s]+)', line)
                if matches:
                    for match in matches:
                        all_extracted_names.append(match.strip())
                else:
                    all_extracted_names.append("NaN")

extracted_df = pd.DataFrame({'EXTRACTED_NAME': all_extracted_names})

# --- Process HCS Files ---
multiple_hcs_files = input("Do you have multiple HCS files to process? (yes/no): ").lower().strip()
all_hcs_data = []

if multiple_hcs_files == 'yes':
    hcs_paths = []
    while True:
        path = input("Please enter the path to an HCS file (or 'done' to finish): ")
        if path.lower() == 'done':
            break
        if os.path.exists(path):
            hcs_paths.append(path)
    for hcs_path in hcs_paths:
        with open(hcs_path, 'r', encoding='utf-8') as file:
            hcs_list = pd.read_html(file.read())
            hcs = hcs_list[0]
            hcs.columns = hcs.iloc[0]
            hcs = hcs[1:].reset_index(drop=True)
            all_hcs_data.append(hcs)
    hcs_df = pd.concat(all_hcs_data, ignore_index=True)
else:
    hcs_path = input("Please enter the path to your HCS file: ")
    if os.path.exists(hcs_path):
        with open(hcs_path, 'r', encoding='utf-8') as file:
            hcs_list = pd.read_html(file.read())
            hcs_df = hcs_list[0]
            hcs_df.columns = hcs_df.iloc[0]
            hcs_df = hcs_df[1:].reset_index(drop=True)

# --- Name Matching ---
hcs_name_col = 'E_NAME'  # HCS Name (E_Name)
account_col = 'ACCOUNT'  # ATM_ACCT
pan_col = 'PAN'  # CARD_NUMBER (adjust if different)
card_code_col = 'CAR_CODE'  # CARD_ID (adjust if different)
expiry_date_col = 'EXPIRYDATE'  # EXPIR_DATE (adjust if different)

# Normalize names
hcs_df['norm'] = hcs_df[hcs_name_col].apply(normalize_name)
extracted_df['norm'] = extracted_df['EXTRACTED_NAME'].apply(normalize_name)

# Create matching keys
hcs_df['match_key'] = hcs_df['norm'].apply(lambda x: f"{x['first']}|{x['middle']}|{x['surname']}")
extracted_df['match_key'] = extracted_df['norm'].apply(lambda x: f"{x['first']}|{x['middle']}|{x['surname']}")

# Merge with extracted_df as base
matched_df = extracted_df.merge(
    hcs_df,
    how='left',
    on='match_key',
    suffixes=('_extracted', '_hcs')
)
matched_df['MATCHED_EXTRACTED_NAME'] = matched_df['EXTRACTED_NAME']

# Keep only rows with HCS matches
matched_df = matched_df.dropna(subset=[hcs_name_col])

# --- Deduplication and Splitting ---
duplicate_mask = matched_df.duplicated(subset=['EXTRACTED_NAME', account_col], keep=False)
duplicate_df = matched_df[duplicate_mask].drop(columns=['norm_extracted', 'norm_hcs', 'match_key'])
unique_df = matched_df[~duplicate_mask].drop(columns=['norm_extracted', 'norm_hcs', 'match_key'])

# --- Compare E_Name and EXTRACTED_NAME in unique_df ---
def check_length_match(row):
    hcs_name = str(row[hcs_name_col]) if pd.notna(row[hcs_name_col]) else ""
    extracted_name = str(row['EXTRACTED_NAME']) if pd.notna(row['EXTRACTED_NAME']) else ""
    return len(hcs_name) == len(extracted_name)

unique_df['length_match'] = unique_df.apply(check_length_match, axis=1)
mismatched_df = unique_df[~unique_df['length_match']].drop(columns=['length_match'])
matched_unique_df = unique_df[unique_df['length_match']].drop(columns=['length_match'])

# --- Generate Excel Output for mismatched_df ---
output_columns = ['ISSTYPE', 'CARD_NUMBER', 'CRDH_NAME', 'ATM_ACCT', 'ISS_DATE', 'EXPIR_DATE', 'CARD_ID']

# Initialize output_df with the same number of rows as mismatched_df
output_df = pd.DataFrame(index=mismatched_df.index, columns=output_columns)

# Populate values
output_df['ISSTYPE'] = 'NEW'  # Changed to uppercase
output_df['CARD_NUMBER'] = mismatched_df[pan_col]
output_df['CRDH_NAME'] = mismatched_df['EXTRACTED_NAME']
output_df['ATM_ACCT'] = mismatched_df[account_col]
output_df['EXPIR_DATE'] = pd.to_datetime(mismatched_df[expiry_date_col]).dt.strftime('%Y-%m-%d')
output_df['ISS_DATE'] = (pd.to_datetime(mismatched_df[expiry_date_col]) - timedelta(days=4*365)).dt.strftime('%Y-%m-%d')
output_df['CARD_ID'] = mismatched_df.get(card_code_col, '')

# --- Output Handling ---
print(f"Current working directory: {os.getcwd()}")
output_dir = input("Enter output directory (press Enter for current directory): ").strip() or os.getcwd()

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

duplicate_path = os.path.join(output_dir, 'duplicate_matches.csv')
mismatched_csv_path = os.path.join(output_dir, 'mismatched_lengths.csv')
matched_unique_path = os.path.join(output_dir, 'matched_unique.csv')
excel_path = os.path.join(output_dir, 'mismatched_output.xlsx')

# Results
print(f"\nOriginal HCS rows: {len(hcs_df)}")
print(f"Extracted names: {len(extracted_df)}")
print(f"Matched rows before splitting: {len(matched_df)}")
print(f"Rows with duplicate ACCOUNT and EXTRACTED_NAME: {len(duplicate_df)}")
print(f"Unique or non-duplicate rows: {len(unique_df)}")
print(f"Rows with mismatched lengths in unique_df: {len(mismatched_df)}")
print(f"Rows with matched lengths in unique_df: {len(matched_unique_df)}")
print("\nFirst few rows of Excel output DataFrame:")
print(output_df.head())

# Save files with error handling
try:
    output_df.to_excel(excel_path, index=False)
    duplicate_df.to_csv(duplicate_path, index=False)
    mismatched_df.to_csv(mismatched_csv_path, index=False)
    matched_unique_df.to_csv(matched_unique_path, index=False)
    print(f"✅ Results saved to:\n- {excel_path}\n- {duplicate_path}\n- {mismatched_csv_path}\n- {matched_unique_path}")
except PermissionError as e:
    print(f"❌ PermissionError: {e}")
    print("Please close any open files or run the script with appropriate permissions.")
except Exception as e:
    print(f"❌ Error saving files: {e}")
    er